In [31]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import drive


In [32]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
image_folder = '/content/drive/MyDrive/scalogram'  # Update this to the path of your image folder
batch_size = 32
image_size = (224, 224)
num_classes = 25

In [34]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [35]:
train_generator = datagen.flow_from_directory(
    image_folder,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 1299 images belonging to 25 classes.


In [36]:
val_generator = datagen.flow_from_directory(
    image_folder,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)



Found 301 images belonging to 25 classes.


In [51]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
#model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
#model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(num_classes, activation='softmax'))

In [56]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [53]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 54, 54, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_11 (Conv2D)          (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_11 (MaxPooli  (None, 26, 26, 128)      

In [57]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
41/41 [==============================] - 183s 4s/step - loss: 3.0623 - accuracy: 0.1255 - val_loss: 3.0154 - val_accuracy: 0.1395
Epoch 2/10
41/41 [==============================] - 179s 4s/step - loss: 3.0156 - accuracy: 0.1339 - val_loss: 3.0682 - val_accuracy: 0.1462
Epoch 3/10
41/41 [==============================] - 177s 4s/step - loss: 2.9978 - accuracy: 0.1594 - val_loss: 2.9137 - val_accuracy: 0.1728
Epoch 4/10
41/41 [==============================] - 176s 4s/step - loss: 2.9167 - accuracy: 0.1824 - val_loss: 2.7785 - val_accuracy: 0.1993
Epoch 5/10
41/41 [==============================] - 171s 4s/step - loss: 2.8238 - accuracy: 0.2086 - val_loss: 2.7579 - val_accuracy: 0.2591
Epoch 6/10
41/41 [==============================] - 179s 4s/step - loss: 2.8196 - accuracy: 0.2009 - val_loss: 2.7937 - val_accuracy: 0.1993
Epoch 7/10
41/41 [==============================] - 178s 4s/step - loss: 2.7658 - accuracy: 0.2286 - val_loss: 2.7947 - val_accuracy: 0.2292
Epoch 8/10
41

In [27]:
test_generator = datagen.flow_from_directory(
    image_folder,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

NameError: name 'y_test_classes' is not defined

In [28]:
model.load_weights('best_model.h5')

10/10 [==============================] - 22s 2s/step - loss: 1.8053 - accuracy: 0.5748
Test Accuracy: 0.5748


In [ ]:
_, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy:.4f}")